# Naturalistic EMA validation

Applying the findings from the 4-state correlation work (EMA x UPDRS) onto real-life EMA data.

Goals:
- analyse real-life variation of EMA values
    - inter-individual variation
    - intra-individual variation, daily fluctuations, differences between days

## 0. Import packages

- document versions for reproducibility

In [ ]:
# import packages
import pandas as pd
import numpy as np
import os
import sys
import csv
import json
import importlib
from itertools import product, compress
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from scipy.signal import welch

In [ ]:
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
# print('mne', mne.__version__)
# print('sci-py', scipy.__version__)
# print('sci-kit learn', sk.__version__)
# print('matplotlib', plt_version)

"""
Python sys 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
pandas 2.1.1
numpy 1.26.0
"""

Import custom functions

In [ ]:
# from current repo
from utils import load_utils, load_data, prep_data


# from naturalistic data repo (home_dbs)
load_utils.add_home_repo()

import load_raw.main_load_raw as load_home
import helper_functions.helpers as home_helpers

## 1. Import Home-Data

Use pre-operative sessions

- use 9-point-converter
- use direct. inverter

In [ ]:
importlib.reload(home_helpers)
importlib.reload(load_home)

data = {}

# Define pre-operative sessions
sel_info = home_helpers.select_sessions()
sel_sessions = {sub: ses for sub, ses in sel_info[['study_id', 'Session']].values}
print(sel_sessions)


for sub, ses in sel_sessions.items():

    data[sub] = load_home.loadSubject(
        sub=sub,
        ses=ses,
        incl_EMA=True,
        incl_ACC=False,
    )



In [ ]:
data['hm12']

## 2. Preprocess data

#### Get (mean-corrected) EMA and UPDRS values per symptom subtype

In [ ]:
figpath = load_utils.get_onedrive_path('emaval_fig')


In [ ]:
import utils.stats as utilsstat
import statsmodels.formula.api as smf
